In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

#from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os
import glob
from torch.utils.tensorboard import SummaryWriter
import random

In [11]:
# get all data
rel_path  = 'Dataset_uptd/all_data'
train_path_stutter = rel_path + '/train_data/wp/'
train_path_fluent = rel_path + '/train_data/fluet/'
test_path_stutter = rel_path + '/train_data/wp/'
test_path_fluent = rel_path + '/train_data/fluent/'

In [12]:
##################################################################################################
# First things first! Set a seed for reproducibility.
# https://www.cs.mcgill.ca/~ksinha4/practices_for_reproducibility/
def set_seed(seed):
    """Set seed"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
##################################################################################################
writer = SummaryWriter()
writer = SummaryWriter("wav2vec_base_model_full_data_wp")
writer = SummaryWriter(comment="Full dataset for binary classification word repetion;")
##################################################################################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device available is', device)
# wav2vec2.0
bundle = torchaudio.pipelines.WAV2VEC2_BASE
print("Sample Rate of model:", bundle.sample_rate)

model_wav2vec = bundle.get_model().to(device)
## Convert audio to numpy to wav2vec feature encodings
def conv_audio_data (filename) :
    waveform, sample_rate = torchaudio.load(filename)
    waveform = waveform.to(device)
    if sample_rate != bundle.sample_rate:
        print('Mismatched sample rate')
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
    emission, _ = model_wav2vec(waveform)
    emission = emission.cpu().detach().numpy()
    return emission

x_f = []
y_f = []
x_s = []
y_s = []

# Convert to the embeddings for training data
for filename in glob.glob(os.path.join(train_path_stutter, '*.wav')):
    stutter_np = conv_audio_data(filename)
    x_s.append(stutter_np)
    y_s.append(1)

for filename in glob.glob(os.path.join(train_path_fluent, '*.wav')):
    fluent_np = conv_audio_data(filename)
    x_f.append(fluent_np)
    y_f.append(1)



Device available is cuda
Sample Rate of model: 16000


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.69 GiB total capacity; 128.99 MiB already allocated; 18.50 MiB free; 140.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
torch.cuda.empty_cache()

In [8]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  132039 KB |  132039 KB |  132039 KB |       0 B  |\n|       from large pool |  131840 KB |  131840 KB |  131840 KB |       0 B  |\n|       from small pool |     199 KB |     199 KB |     199 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  132039 KB |  132039 KB |  132039 KB |       0 B  |\n|       from large pool |  131840 KB |  131840 KB |